In [1]:
# Installing tf-nightly to have EfficientNets
!pip install tf-nightly-gpu
# check version
import tensorflow as tf
tf.__version__

     |████████████████████████████████| 523.8MB 35kB/s 
     |████████████████████████████████| 460kB 2.5MB/s 
     |████████████████████████████████| 2.9MB 12.9MB/s 


'2.3.0-dev20200527'

In [2]:
#Tensorflow Import and GPU recognition
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7934996009978370810, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10653193085509983251
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9255599881510938357
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15681750336
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8146702156669217960
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
!unzip '/content/drive/My Drive/Progetto Advanced Machine Learning/fruits.zip' -d '/content'

Archive:  /content/drive/My Drive/Progetto Advanced Machine Learning/fruits.zip
replace /content/fruits-360_dataset/fruits-360/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [6]:
#Importing Keras and other useful libs
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
import tensorflow.keras as keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from glob import glob
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [7]:
#Defining dataset path and saving it for future usage
path = '/content/fruits-360_dataset/fruits-360'
os.listdir(path)

['test-multiple_fruits', 'papers', 'Test', 'Training', 'LICENSE', 'readme.md']

In [8]:
#Number of pictures in Training folder
training_files = glob(os.path.join(path,'Training', '*/*.jpg'))
image_num = len(training_files)
print("Number of Images: ",image_num)

Number of Images:  60498


In [9]:
#Number of pictures in Test Folder
testing_files = glob(os.path.join(path, 'Test', '*/*.jpg'))
img_num = len(testing_files)
print("Number of Images: ", img_num)

Number of Images:  20622


In [10]:
#Print category details
image_count = []
class_names = []
print('{:18s}'.format('Class'), end='')
print('Count:')
print('-'*24)
for folder in os.listdir(os.path.join(path,'Training')):
  folder_count = len(os.listdir(os.path.join(path,'Training',folder)))
  image_count.append(folder_count)
  class_names.append(folder)
  print('{:20s}'.format(folder), end='')
  print(folder_count)
print('-'*24)
print('Number of Classes:', len(class_names))
print('Average number of images per Class: ', np.array(image_count).mean())

Class             Count:
------------------------
Apple Braeburn      492
Rambutan            492
Orange              479
Potato Red          450
Cantaloupe 1        492
Beetroot            450
Mulberry            492
Apple Red 3         429
Onion Red Peeled    445
Pear Williams       490
Cactus fruit        490
Pear Forelle        702
Kohlrabi            471
Pear Red            666
Carambula           490
Tomato 4            479
Lychee              490
Pear Kaiser         300
Apple Red 2         492
Grape White         490
Pomelo Sweetie      450
Pepper Yellow       666
Potato Sweet        450
Tomato 1            738
Passion Fruit       490
Pear Abate          490
Apple Red Yellow 2  672
Grapefruit White    492
Grape Pink          492
Walnut              735
Tamarillo           490
Cocos               490
Peach Flat          492
Chestnut            450
Pitahaya Red        490
Apple Red 1         492
Pear                492
Pineapple Mini      493
Cherry 1            492
Grape White 4 

In [11]:
#Definitive paths
train_out_path = os.path.join(path,'Training')
test_out_path = os.path.join(path, 'Test')
print(train_out_path)
print(test_out_path)

/content/fruits-360_dataset/fruits-360/Training
/content/fruits-360_dataset/fruits-360/Test


In [0]:
#2 Approach: Inception v3 pre-trained
from keras_applications.vgg16 import VGG16
from keras_applications.vgg16 import preprocess_input
from keras.optimizers import Adam
from keras import optimizers
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras import regularizers
from keras.layers.convolutional import *
from keras.models import Model

In [0]:
#Implementing some augmentation to avoid overfitting on the training generator
train_datagenerator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)
train_and_val_generator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True)
test_datagenerator = ImageDataGenerator(rescale= 1./255)

In [14]:
#Creating Batches
image_size = (80, 80)
train_batches = train_datagenerator.flow_from_directory(train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='training', seed = 20052020)
val_batches = train_datagenerator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='validation', seed = 20052020)
train_val_batches = train_and_val_generator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, seed = 20052020)
test_batches = test_datagenerator.flow_from_directory(directory=test_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, shuffle=False)

Found 48431 images belonging to 120 classes.
Found 12067 images belonging to 120 classes.
Found 60498 images belonging to 120 classes.
Found 20622 images belonging to 120 classes.


In [0]:
efficient3_model = tf.keras.applications.EfficientNetB7(include_top=False, weights='imagenet', input_shape=image_size+(3,))
#Using pretrained
for layer in efficient3_model.layers:
  layer.trainable=False
#Adding Layers at the top
top = efficient3_model.output
top = tf.keras.layers.GlobalAveragePooling2D('channels_last')(top)
#top = tf.keras.layers.Dropout(0.2, name='top_dropout')(top)


#Adding Classification layer with softmax activation function(needed for categorical predictions)
predictor = tf.keras.layers.Dense(120 ,activation='softmax', name='predictor')(top)
efficient3_model = tf.keras.Model(inputs=efficient3_model.input, outputs=predictor)

In [0]:
#efficient3_model.get_layer('top_conv').trainable = True
#efficient3_model.get_layer('top_bn').trainable = True
for layer in efficient3_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

In [27]:
efficient3_model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 80, 80, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization_2 (Normalization) (None, 80, 80, 3)    7           rescaling_2[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 81, 81, 3)    0           normalization_2[0][0]            
_______________________________________________________________________________________

In [0]:
#Compiling the model
efficient3_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss='categorical_crossentropy', metrics=['accuracy'])
efficient3_model.save_weights('Initial weights.h5')

In [0]:
#Training and Validating the model: Round 1
wait = 5
epochs = 15
model_name='EfficientNetB3'
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=wait)

In [33]:
efficient3_model.fit_generator(train_batches, epochs=5 , verbose=1, shuffle=True, validation_data=val_batches, callbacks=[early_stopping])
efficient3_model.load_weights('Initial weights.h5')
trainingMdl = efficient3_model.fit_generator(train_val_batches, epochs=early_stopping.stopped_epoch , verbose=1, shuffle=True, validation_data=val_batches)

Epoch 1/5
 461/1514 [========>.....................] - ETA: 1:29 - loss: 5.3996 - accuracy: 0.0074

KeyboardInterrupt: ignored

In [0]:
import sys
sys.path.append('/content/drive/My Drive/Progetto Advanced Machine Learning/')
from utils import plot_history
history_fig = plot_history(trainingMdl)
history_fig.show()

ValueError: ignored